In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sqlalchemy import create_engine
import matplotlib as mpl
import numpy as np
import seaborn as sns
from scipy import stats
from sklearn import preprocessing
%matplotlib inline

In [2]:
path = r"XXXX-XXXX"
con = sqlite3.connect(path)
cur = con.cursor()
midfielder = pd.read_sql_query("SELECT *  FROM avgPlayerStatistics AS ps WHERE ps.player_position = 'M' ;", con,)


In [4]:
pd.set_option('display.max_columns', None)
midfielder.head()

,index,assists,blocks,commited_fouls,commited_penalty,conceded,drawn_fouls,dribble_attempts,dribble_past,dribble_success,fixture_id,interceptions,key_passes,minutes,missed_penalty,number,offside,on_goal_shots,pass_accuracy,player_id,player_position,rating,red,saved_penalty,saves,scored_penalty,timestamp,total_duels,total_goals,total_passes,total_shots,total_tackels,won_duels,won_penalty,yellow
0,2,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0,185545,1.0,1.0,90.0,0.0,2.0,0.0,1.0,80.0,560,M,6.6,0.0,0.0,0.0,0.0,1438972200,11.0,0.0,21.0,2.0,4.0,5.0,0.0,0.0
1,5,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,185545,3.0,0.0,90.0,0.0,4.0,0.0,0.0,84.0,21706,M,7.0,0.0,0.0,0.0,0.0,1438972200,18.0,0.0,54.0,2.0,3.0,8.0,0.0,0.0
2,6,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,185545,1.0,0.0,68.0,0.0,24.0,0.0,0.0,92.0,87082,M,6.9,0.0,0.0,0.0,0.0,1438972200,3.0,0.0,38.0,0.0,0.0,3.0,0.0,0.0
3,7,0.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,1.0,185545,3.0,0.0,90.0,0.0,9.0,0.0,0.0,74.0,7011,M,6.6,0.0,0.0,0.0,0.0,1438972200,15.0,0.0,29.0,1.0,4.0,7.0,0.0,0.0
4,9,0.0,0.0,0.0,0.0,0.0,7.0,8.0,0.0,4.0,185545,0.0,1.0,90.0,0.0,7.0,0.0,0.0,69.0,2710,M,7.0,0.0,0.0,0.0,0.0,1438972200,23.0,0.0,25.0,2.0,3.0,15.0,0.0,1.0


In [47]:
df = midfielder[['fixture_id','player_id','key_passes','dribble_success', 'pass_accuracy', 'total_passes', 'won_duels', 'drawn_fouls', 'dribble_attempts', 'assists' ,'total_duels', 'interceptions', 'blocks', 'minutes','total_goals','total_shots', 'on_goal_shots' ]]
#display(df)
df.head()

,fixture_id,player_id,key_passes,dribble_success,pass_accuracy,total_passes,won_duels,drawn_fouls,dribble_attempts,assists,total_duels,interceptions,blocks,minutes,total_goals,total_shots,on_goal_shots
0,185545,560,1.0,0.0,80.0,21.0,5.0,1.0,1.0,0.0,11.0,1.0,0.0,90.0,0.0,2.0,1.0
1,185545,21706,0.0,2.0,84.0,54.0,8.0,2.0,2.0,0.0,18.0,3.0,0.0,90.0,0.0,2.0,0.0
2,185545,87082,0.0,1.0,92.0,38.0,3.0,1.0,1.0,0.0,3.0,1.0,0.0,68.0,0.0,0.0,0.0
3,185545,7011,0.0,1.0,74.0,29.0,7.0,1.0,4.0,0.0,15.0,3.0,0.0,90.0,0.0,1.0,0.0
4,185545,2710,1.0,4.0,69.0,25.0,15.0,7.0,8.0,0.0,23.0,0.0,0.0,90.0,0.0,2.0,0.0


In [48]:
for key in df.keys(): # check for equality of dataframes
    print(key,":", midfielder[key].equals(df[key]))

fixture_id : True
player_id : True
key_passes : True
dribble_success : True
pass_accuracy : True
total_passes : True
won_duels : True
drawn_fouls : True
dribble_attempts : True
assists : True
total_duels : True
interceptions : True
blocks : True
minutes : True
total_goals : True
total_shots : True
on_goal_shots : True


In [49]:
# NORMALIZE ALLE AUßER FIXTURE_ID

x = df.to_numpy() #returns a numpy array
x1 = x[:, 2:] # every column from colum (player_id)


robust_scaler = preprocessing.RobustScaler()
x_scaled = robust_scaler.fit_transform(x1)
x_scaled = np.concatenate(( x[:,[0,1]], x_scaled), axis=1)
final = pd.DataFrame(x_scaled, columns=df.columns)
columns = df.keys()
for i in range(2): # fixture_id bis timestamp zu int
    final[columns[i]] = pd.to_numeric(final[columns[i]], downcast='integer')
display(final)


,fixture_id,player_id,key_passes,dribble_success,pass_accuracy,total_passes,won_duels,drawn_fouls,dribble_attempts,assists,total_duels,interceptions,blocks,minutes,total_goals,total_shots,on_goal_shots
0,185545,560,0.2,-0.75,0.177215,-0.303030,0.3125,0.0,-0.142857,0.0,0.464286,0.2,0.0,0.473684,0.0,1.2,1.333333
1,185545,21706,-0.8,1.75,0.430380,1.363636,1.2500,1.0,0.571429,0.0,1.714286,2.2,0.0,0.473684,0.0,1.2,-0.333333
2,185545,87082,-0.8,0.50,0.936709,0.555556,-0.3125,0.0,-0.142857,0.0,-0.964286,0.2,0.0,-0.353383,0.0,-0.8,-0.333333
3,185545,7011,-0.8,0.50,-0.202532,0.101010,0.9375,0.0,2.000000,0.0,1.178571,2.2,0.0,0.473684,0.0,0.2,-0.333333
4,185545,2710,0.2,4.25,-0.518987,-0.101010,3.4375,6.0,4.857143,0.0,2.607143,-0.8,0.0,0.473684,0.0,1.2,-0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120778,705683,3325,0.6,0.50,-4.050633,-0.505051,0.0000,1.2,0.571429,0.0,0.142857,-0.4,0.0,0.338346,0.0,0.0,0.333333
120779,705683,24860,1.2,2.50,-3.620253,-0.020202,0.4375,0.2,2.285714,0.0,0.250000,0.0,0.0,0.000000,1.0,1.4,2.000000
120780,705683,41500,-0.8,0.00,-2.658228,-1.131313,-0.7500,0.4,0.142857,0.0,-1.035714,-0.6,0.0,-1.947368,0.0,-0.2,0.333333
120781,705683,15327,-0.2,-0.75,-0.367089,0.090909,-0.3750,0.0,-0.857143,1.0,-0.535714,0.4,0.0,-0.571429,0.0,-0.4,-0.333333


In [15]:
#NORMALIZE ALL FOR COMPARISON

x = df.to_numpy() #returns a numpy array
robust_scaler = preprocessing.RobustScaler()
x_scaled = robust_scaler.fit_transform(x)
all_norm = pd.DataFrame(x_scaled, columns=df.columns)

In [16]:
#TESTING FINALDATASET FOR FIXTURE ORDER

columns = df.keys()
print(columns[0]) #check für index 0,1,2 ist per hand passiert
x = columns[0]
final[x] = pd.to_numeric(final[x], downcast='integer')
df[x] = pd.to_numeric(df[x], downcast='integer')
print(df[x].equals(final[x]))  # Returns True)
print(type(columns))

#TEST 
# haben fixture_id bis timestamp gleiche Reihenfolge
# wenn ja sind in richtiger reiheinfolge konkateniert

fixture_id
True
<class 'pandas.core.indexes.base.Index'>


C:\Users\Anwender\AppData\Local\Temp/ipykernel_992/3261938473.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[x] = pd.to_numeric(df[x], downcast='integer')


In [50]:
for key in df.keys():
    print(key,":", all_norm[key].equals(final[key]))

fixture_id : False
player_id : False
key_passes : True
dribble_success : True
pass_accuracy : True
total_passes : True
won_duels : True
drawn_fouls : True
dribble_attempts : True
assists : True
total_duels : True
interceptions : True
blocks : True
minutes : True
total_goals : True
total_shots : True
on_goal_shots : True


In [84]:
weights = [1,1,0.15,0.1,0.1,0.1,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05]
print(len(weights))
final_mul_weights = final.mul(weights)
final_mul_weights['fixture_id'] = pd.to_numeric(final_mul_weights['fixture_id'], downcast='integer')
final_mul_weights['player_id'] = pd.to_numeric(final_mul_weights['player_id'], downcast='integer')  #um equals true zu erreichen müssen fixture id gleichen typ haben
print("fixture_id:", final['fixture_id'].equals(final_mul_weights['fixture_id']))
print("player_id:", final['player_id'].equals(final_mul_weights['player_id']))
final_mul_weights.head()



17
fixture_id: True
player_id: True


,fixture_id,player_id,key_passes,dribble_success,pass_accuracy,total_passes,won_duels,drawn_fouls,dribble_attempts,assists,total_duels,interceptions,blocks,minutes,total_goals,total_shots,on_goal_shots
0,185545,560,0.03,-0.075,0.017722,-0.030303,0.015625,0.00,-0.007143,0.0,0.023214,0.01,0.0,0.023684,0.0,0.06,0.066667
1,185545,21706,-0.12,0.175,0.043038,0.136364,0.062500,0.05,0.028571,0.0,0.085714,0.11,0.0,0.023684,0.0,0.06,-0.016667
2,185545,87082,-0.12,0.050,0.093671,0.055556,-0.015625,0.00,-0.007143,0.0,-0.048214,0.01,0.0,-0.017669,0.0,-0.04,-0.016667
3,185545,7011,-0.12,0.050,-0.020253,0.010101,0.046875,0.00,0.100000,0.0,0.058929,0.11,0.0,0.023684,0.0,0.01,-0.016667
4,185545,2710,0.03,0.425,-0.051899,-0.010101,0.171875,0.30,0.242857,0.0,0.130357,-0.04,0.0,0.023684,0.0,0.06,-0.016667


In [85]:
fixtures = final_mul_weights.pop('fixture_id')
players = final_mul_weights.pop('player_id')


In [86]:
fixtures.head()
# players.head()
# final_mul_weights.head()
# display(final_mul_weights)

0    185545
1    185545
2    185545
3    185545
4    185545
Name: fixture_id, dtype: int32

In [87]:
test = final_mul_weights.sum(axis=1)
display(test)
test.head()
player_rating = pd.Series(test,
              name="m_rating")
player_rating = player_rating.to_frame()
print(type(player_rating))

0         0.134466
1         0.638205
2        -0.056092
3         0.252669
4         1.265107
            ...   
120778   -0.206270
120779    0.444615
120780   -0.681799
120781   -0.216249
120782   -0.454073
Length: 120783, dtype: float64

<class 'pandas.core.frame.DataFrame'>


In [88]:
player_rating.insert(0, 'fixture_id', fixtures)
player_rating.insert(1, 'player_id', players)

In [89]:
#TESTING FINALDATASET FOR FIXTURE ORDER

columns = df.keys()
print(columns[0]) #check für index 0,1,2 ist per hand passiert
x = columns[0]
player_rating[x] = pd.to_numeric(player_rating[x], downcast='integer')
df[x] = pd.to_numeric(df[x], downcast='integer')
print(df[x].equals(player_rating[x]))  # Returns True)
print(type(columns))

#TEST 
# haben fixture_id bis timestamp gleiche Reihenfolge
# wenn ja sind in richtiger reiheinfolge konkateniert

fixture_id
True
<class 'pandas.core.indexes.base.Index'>


C:\Users\Anwender\AppData\Local\Temp/ipykernel_992/2261603312.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[x] = pd.to_numeric(df[x], downcast='integer')


In [90]:
player_rating.describe()
pd.get_option("display.max_rows")
pd.set_option("display.max_rows", 20)

In [91]:
engine = create_engine(r'XXXX-XXXX', echo=False)
player_rating.to_sql('midfieldRating', con=engine, if_exists='append') #if_exists='append'


In [92]:
test = pd.read_sql_query("SELECT DISTINCT player_id FROM midfieldRating ORDER BY m_rating DESC LIMIT 10;", con)
display(test)

,player_id
0,297
1,1821
2,2414
3,865
4,37127
5,634
6,635
7,47390
8,154
9,2057


In [96]:
test = pd.read_sql_query("SELECT  * FROM lineups WHERE player_id =37127 AND player_position = 'M';", con)
display(test)

,fixture_id,team_id,timestamp,player_id,player_name,player_number,player_position,grid
0,592726,42,1613320200,37127,M. Ødegaard,11,M,3:1
1,592736,42,1613925000,37127,M. Ødegaard,11,M,4:3
2,592758,42,1615033800,37127,M. Ødegaard,11,M,4:3
3,592766,42,1615739400,37127,M. Ødegaard,11,M,4:3
4,592785,42,1616338800,37127,M. Ødegaard,11,M,4:3
...,...,...,...,...,...,...,...,...
40,3432,200,1557664200,37127,M. Ødegaard,18,M,3:2
41,150613,200,1558723500,37127,M. Ødegaard,18,M,4:3
42,150614,200,1559069100,37127,M. Ødegaard,18,M,3:1
43,150617,200,1558205100,37127,M. Ødegaard,18,M,4:1
